In [1]:
try:
    from selenium import webdriver
except ImportError:
    print("selenium is not installed. Installing now...")
    !pip install selenium
    from selenium import webdriver
    
try:
    from lxml import html
except ImportError:
    print("lxml is not installed. Installing now...")
    !pip install lxml
    from lxml import html
    
try:
    from tqdm import tqdm
except ImportError:
    print("tqdm is not installed. Installing now...")
    !pip install tqdm
    from tqdm import tqdm

try:
    import requests
except ImportError:
    print("tqdm is not installed. Installing now...")
    !pip install requests
    import requests

print("All required libraries are installed.")

selenium is not installed. Installing now...
  Using cached selenium-4.11.2-py3-none-any.whl (7.2 MB)
  Using cached trio-0.22.2-py3-none-any.whl (400 kB)
  Using cached trio_websocket-0.10.3-py3-none-any.whl (17 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached exceptiongroup-1.1.3-py3-none-any.whl (14 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
All required libraries are installed.


In [2]:
import time
import os
import re
from collections import OrderedDict
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [ ]:
# 自己的 cookie
cookie = ''

In [3]:
# https://www.tiktok.com/@_lx0606__
# https://www.tiktok.com/@sooflower_
# 填入想要爬取的頁面
url = "https://www.tiktok.com/@boon.ong"
max_threads = 10

headers = {
    'origin': 'https://www.tiktok.com',
    'referer': 'https://www.tiktok.com/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'cookie': cookie,
}

In [4]:
def get_author_information(url,headers):
    html_content = requests.get(url,headers).text
    tree = html.fromstring(html_content)
    author = tree.cssselect("div h2")[0].text_content().replace(" ", "")
    authorID = tree.cssselect("div h1")[0].text_content().replace(" ", "")
    return author, authorID

In [5]:
def get_video_links(url):
    links = []
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get(url)
    # 等待一些時間，讓頁面完全加載
    while True:
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        current_html = driver.page_source
        # 等待一些時間，讓下滑後的內容加載
        time.sleep(2)
        tree = html.fromstring(current_html)
        linksElements = tree.cssselect(".tiktok-16ou6xi-DivTagCardDesc.eih2qak1 a:nth-child(1)")
        for element in linksElements:
            href = element.attrib.get("href")
            links.append(href)
        # 檢查是否可以繼續下滑，如果不行則退出迴圈
        previous_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        current_height = driver.execute_script("return document.body.scrollHeight")
        if current_height <= previous_height:
            cu = input("Do you want to continue ? Y / N")
            if cu.lower() == 'y':
                continue
            else:
                break     
    
    driver.quit()
    
    filtered_links = [link for link in links if 'https://www.tiktok.com/' in link]
    return list(OrderedDict.fromkeys(filtered_links))


In [6]:
author, authorID = get_author_information(url,headers)

In [7]:
if not os.path.isdir(author):
    os.mkdir(author)
    print('資料夾名稱為', author, '正在幫您建立')

資料夾名稱為 분옹♥ 正在幫您建立


In [8]:
video_links = get_video_links(url)

In [9]:
def get_download_link_and_title(url):
    try:
        html_content = requests.get(url, headers = {
            'cookie': cookie,
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
        }).text
        video_title_re = '"og:description":"(.*?)","fb:app_id"'
        video_download_link_re = '{"url":"(.*?)","id":""}'
        video_title = re.findall(video_title_re, html_content)[0]
        video_download_link = re.findall(video_download_link_re, html_content)[0]
        video_download_link = re.sub(r'\\u0026', '&', video_download_link)
        video_download_link = re.sub(r'\\u002F', '/', video_download_link)
        return video_download_link, video_title
    except Exception as e:
        print(f'出現錯誤: \n{e}')

In [10]:
def sanitize_filename(filename):
    sanitized_filename = re.sub(r'[\/:*?"<>|]', '_', filename)
    max_filename_length = 100 
    if len(sanitized_filename) > max_filename_length:
        sanitized_filename = sanitized_filename[:max_filename_length]
    return sanitized_filename

In [11]:
def process_link(url):
    try:
        download_link, title = get_download_link_and_title(url)
        return download_link, title
    except Exception as e:
        print(f"Error processing '{url}': {e}")
        return None, None

In [12]:
def download_video(url,filename):
    try:
        vidal = requests.get(url,headers={
            'cookie': cookie,
            'referer': 'https://www.tiktok.com/',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        }).content
        with open(filename, mode='wb') as f:
            f.write(vidal)
            print(f'已下載完成 :{filename}')
    except Exception as e:
        print(f"Error processing '{url}': {e}")

In [13]:
titles = []
download_links = []
with ThreadPoolExecutor(max_threads) as executor, tqdm(total=len(video_links), desc="Processing videos", unit="video") as progress:
    futures = [executor.submit(process_link, url) for url in video_links]
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        progress.update(1)
        if result[0] is not None:
            download_links.append(result[0])
        if result[1] is not None:
            titles.append(result[1])

Processing videos: 100%|██████████| 139/139 [00:15<00:00,  8.94video/s]


In [14]:
with ThreadPoolExecutor(max_threads) as executor, tqdm(total=len(download_links), desc="Downloading videos", unit="video") as progress:
    download_args = zip(download_links, titles)
    futures = [executor.submit(download_video, link, f'{author}\\{sanitize_filename(title)}.mp4') for link, title in download_args]
    for future in concurrent.futures.as_completed(futures):
        progress.update(1)

已下載完成 :분옹♥\がんばれ！ . . . #boonong #분옹 #분옹♥ #fypシ #GambareGambareSenpai #BEMAX #간바레 #간바레챌린지 .mp4


已下載完成 :분옹♥\킥 드럼 베이스!!  . . . #boonong #분옹 #분옹♥ #fyp #킥드럼베이스 #댄스챌린지 #challenge #dancechallenge #챌린지 .mp4
已下載完成 :분옹♥\Feeling Good . . . #boonong #분옹 #분옹❤️  #FeelingGood #MichaelBublé #fyp #전신 #전신샷 #outd #스타킹 #검정스타킹 #전.mp4


已下載完成 :분옹♥\just being honest . . . #boonong #분옹 #분옹♥ #fyp  #LikeGirls feat. #Tyga - #Pleasure P#оригинальныйзву.mp4
已下載完成 :분옹♥\Trend qua chưa zị  . . . #boonong #분옹 #분옹♥ #fyp  #YouAre #JinD #Remix #JayDMuzik #xuhuong #xuhuongti.mp4
已下載完成 :분옹♥\Mộng mơ kia, không giữ lâu . . . #boonong #분옹 #분옹♥ #España #NhữngLờiDốiGian #Remix #LalaTrần #fyp #o.mp4


已下載完成 :분옹♥\안녕! 난 강백호 여친 🙊 . . . #Boonong #분옹 #분옹❤️ #fyp #ootd #치어리더 #유니폼 #농구복 #슬램덩크 #강백호 #여친룩 .mp4
已下載完成 :분옹♥\Bài hát rất đáng yêu . . . #boonong #분옹 #분옹♥ #fyp  #xuhuong #xuhuongtiktok  #NgõChạm #BigDaddy #Emil.mp4


已下載完成 :분옹♥\15 minuto de sentada, faço você se iludir . . . #Boonong #분옹 #분옹♥   #NenoNobeat & #MCRica #VemMoreni.mp4


...

已下載完成 :분옹♥\🦭 #Boonong #분옹♥   #fyp #fotyou #fotyoupage #viral  #vlog #OOTD #레깅스   #틱톡 #추천 #추천떠라 #추천떠라얍 #귀여운 #애교 .mp4
已下載完成 :분옹♥\#cureforme 🤧  .  .  .  #Boonong #분옹♥   #fyp #fotyou #fotyoupage #viral  #vlog #OOTD #레깅스  #틱톡 #추천 #추.mp4
已下載完成 :분옹♥\내 주사는.... 있어요 그런게 🙈  . . . #burningfridaynight 🍻 #Boonong #분옹♥     #fyp #fotyou #fotyoupage #viral  .mp4
已下載完成 :분옹♥\빵야 빵야 🔫  .  .  .  #Boonong #분옹♥   #fyp #fotyou #fotyoupage #viral  #vlog #OOTD #japen  #틱톡 #추천 #추천떠라.mp4


已下載完成 :분옹♥\🥀 .  .  .  #Boonong #분옹♥   #fyp #fotyou #fotyoupage #viral  #vlog #OOTD #việtnam  #틱톡 #추천 #추천떠라 #추천떠.mp4


已下載完成 :분옹♥\누나도 네모춤 가능!! 🙃..._  .  .  .  #Boonong #분옹♥   #fyp #fotyou #fotyoupage #viral  #vlog #OOTD    #틱톡 #추천.mp4


已下載完成 :분옹♥\안돼!! 🤨  .  .  .  #Boonong #분옹♥   #fyp #fotyou #fotyoupage #viral  #vlog #OOTD #japan  #だめ(다메) #다메 #다.mp4


已下載完成 :분옹♥\귀여워 해줘 😘 #Boonong #분옹♥   #fyp #fotyou #fotyoupage #viral  #vlog #OOTD #이모지카메라 #틱톡 #추천 #추천떠라 #추천떠라얍  .mp4


影片合併

In [15]:
import subprocess
import os

In [16]:
current_directory = os.getcwd()
video_dir = f'./{author}'
videolist_file = os.path.join(current_directory, "videolist.txt")
output_file = os.path.join(current_directory, f'./{author}.mp4')
extension = ".mp4"

In [17]:
video_files = [file for file in os.listdir(video_dir) if file.endswith(extension)]

videolist_contents = "\n".join([f"file '{os.path.join(video_dir, file)}'" for file in video_files])

with open(videolist_file, "w", encoding="utf-8") as f:
    f.write(videolist_contents)

In [18]:
ffmpeg_command = [
    "ffmpeg",
    "-f", "concat",
    "-safe", "0",
    "-i", videolist_file,
    "-c", "copy",
    output_file
]

try:
    subprocess.run(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True, encoding='utf-8')
    print("Video merged successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error merging videos: {e.stderr}")

# 合并完成后删除 videolist_file
try:
    os.remove(videolist_file)
    print(f"Deleted {videolist_file}")
except Exception as e:
    print(f"Error deleting {videolist_file}: {e}")
    
# 轉 python 
# pip install jupyter
# pip install nbconvert
# python -m nbconvert --to script TikTok.ipynb

Video merged successfully.
Deleted c:\code\pythone\國際板抖音爬蟲\videolist.txt
